In [5]:
import pickle
# import omegaconf
import warnings
import pickle
import numpy as np
warnings.filterwarnings("ignore", category=DeprecationWarning)

project_name = 'full_fine_tuning_5epochs_article1'

#
datas=( 'uc-merced-land-use-dataset',
       'stanford_cars', 
       'caltech101', 
       'fgvc-aircraft-2013b', 
       'flowers-102',
         'oxford-iiit-pet'  ) #'dtd',

losses=( 'TRADES_v2', 'CLASSIC_AT' ) # 

model_parameters = {
    'convnext_base': 86.0,
    'convnext_tiny': 28.0,
    'deit_small': 22.0,
    'vit_base': 86.0,
    'vit_small': 22.0,
    'resnet50': 25.0,
    'eva02_base': 78.0,
    'eva02_tiny': 24.0,
    'swin_base': 88.0,
    'swin_tiny': 28.0,
    'coatnet_0': 33.0,
    'coatnet_2': 77.0
}

# Two backbone groups
backbones=(
  'CLIP-convnext_base_w-laion_aesthetic-s13B-b82K',
  'CLIP-convnext_base_w-laion2B-s13B-b82K',
  'deit_small_patch16_224.fb_in1k',
  'robust_resnet50',
  'vit_small_patch16_224.augreg_in21k',
  'convnext_base.fb_in1k',
  'resnet50.a1_in1k',
  'robust_vit_base_patch16_224',
  'vit_base_patch16_224.mae',
  'vit_small_patch16_224.dino',
  'convnext_base.fb_in22k',

  'robust_convnext_base',
  'vit_base_patch16_224.augreg_in1k',
  'vit_base_patch16_224.augreg_in21k',
  'vit_base_patch16_clip_224.laion2b',
  'convnext_tiny.fb_in1k',
  'robust_convnext_tiny',
  'robust_deit_small_patch16_224',
  'vit_small_patch16_224.augreg_in1k',
  'convnext_tiny.fb_in22k',
  'vit_base_patch16_clip_224.laion2b_ft_in1k',
  'vit_base_patch16_224.augreg_in21k_ft_in1k',

  'vit_small_patch16_224.augreg_in21k_ft_in1k',
  'eva02_base_patch14_224.mim_in22k',
  'eva02_tiny_patch14_224.mim_in22k',
  'swin_base_patch4_window7_224.ms_in22k_ft_in1k',
  'swin_tiny_patch4_window7_224.ms_in1k',
  'convnext_base.clip_laion2b_augreg_ft_in12k_in1k',
  'convnext_base.fb_in22k_ft_in1k',
  'convnext_tiny.fb_in22k_ft_in1k',
  'coatnet_0_rw_224.sw_in1k',
  'coatnet_2_rw_224.sw_in12k_ft_in1k',
  'coatnet_2_rw_224.sw_in12k',
)

import math
from typing import Mapping, Tuple

def sums_from_dict(scores):

    values = []
    test=False
    for v in scores.values():
        if v is None or (isinstance(v, float) and math.isnan(v)):
            test=True
        values.append(float(v))

    if test:
        return math.nan, math.nan

    arith_sum = sum(values)
    geom_sum  = math.prod(values)            # built‑in in Python ≥3.8
    return arith_sum, geom_sum

final_data = []

for loss in losses:
    for data in datas:
        for backbone in backbones:

            try:
                name ='{}_{}_{}'.format(backbone, data, loss)
                print( '../results/{}/{}.pkl'.format(project_name, name) )
                with open('../results/{}/{}.pkl'.format(project_name, name), 'rb') as f:
                    result = pickle.load(f)
                                    # result = saved_data[name]["statistics"]

                arith_sum, geom_sum = sums_from_dict(result)
                result['sum'] = arith_sum
                result['geom'] = geom_sum

            except:
                
                result = {'clean_acc': None, 'Linf_acc': None, 'L2_acc': None, 'L1_acc': None, 'common_acc': None, 
                          'sum':None, 'geom':None, }
                
            for key, value in model_parameters.items():
                if key in backbone:  # Match the model name in the backbone string
                    result['model_size'] = 1 if value < 50 else 2
                    break

            result['backbone'] = backbone
            result['dataset'] = data
            result['loss_function'] = loss  
            final_data.append(result)

../results/full_fine_tuning_5epochs_article1/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_5epochs_article1/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_5epochs_article1/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_5epochs_article1/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_5epochs_article1/vit_small_patch16_224.augreg_in21k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_5epochs_article1/convnext_base.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_5epochs_article1/resnet50.a1_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_5epochs_article1/robust_vit_base_patch16_224_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/full_fine_tuning_5epochs_article1/vit_base_patch16_224.mae_uc-merc

In [11]:
# Pivot the table to group by 'backbone' and 'loss_function' in rows and create a hierarchical structure with 'dataset' as column index

# df['total_acc'] = df['clean_acc'] + df['Linf_acc']  # Sum of scores per dataset

import pandas as pd

df = pd.DataFrame(final_data)

nan_percentage = (df.isna().sum().sum() / df.size) * 100
print(f"Percentage of NaN values: {nan_percentage:.2f}%")

df = df[ df.model_size == 1 ]

grouped_df = df.pivot_table(
    index=['backbone', 'loss_function'],
    columns='dataset',
    # dropna=False
)

# Rename column levels
grouped_df.columns.set_names(["metric", "dataset"], inplace=True)

# Swap column levels → dataset becomes level 0, metric becomes level 1
grouped_df.columns = grouped_df.columns.swaplevel(0, 1)

# (Optional) Sort columns so that all metrics are grouped within each dataset
grouped_df = grouped_df.sort_index(axis=1, level=0)

grouped_df.to_csv("./{}.csv".format(project_name) )

# --- existing shortcuts ----------------------------------------------------
sum_scores  = grouped_df.xs('sum',  level=1, axis=1)     # (rows × 6 datasets)
geom_scores = grouped_df.xs('geom', level=1, axis=1)     # (rows × 6 datasets)

#--- 1. add the TOTAL aggregates you already computed ----------------------
grouped_df[('TOTAL', 'score_sum')]  = sum_scores.sum(axis=1)
grouped_df[('TOTAL', 'score_geom')] = geom_scores.sum(axis=1)

# --- 2. count NaNs across datasets -----------------------------------------
grouped_df[('TOTAL', 'nan_sum_cnt')]  = sum_scores.isna().sum(axis=1)
grouped_df[('TOTAL', 'nan_geom_cnt')] = geom_scores.isna().sum(axis=1)

# --- 3. rank as before ------------------------------------------------------
df_sorted = grouped_df.sort_values(('TOTAL', 'score_sum'), ascending=False)

df_sorted

# df_sorted.to_csv("./df_sorded_{}.csv".format(project_name) )


Percentage of NaN values: 4.71%


dataset                                                  caltech101            \
metric                                                       L1_acc    L2_acc   
backbone                                   loss_function                        
robust_convnext_tiny                       CLASSIC_AT      0.502000  0.671000   
robust_resnet50                            CLASSIC_AT      0.140000  0.427000   
robust_convnext_tiny                       TRADES_v2       0.307000  0.698000   
swin_tiny_patch4_window7_224.ms_in1k       TRADES_v2       0.342000  0.152000   
convnext_tiny.fb_in1k                      TRADES_v2       0.000000  0.000000   
deit_small_patch16_224.fb_in1k             TRADES_v2       0.421333  0.375000   
robust_resnet50                            TRADES_v2       0.088000  0.114000   
deit_small_patch16_224.fb_in1k             CLASSIC_AT      0.548000  0.538000   
robust_deit_small_patch16_224              CLASSIC_AT      0.153000  0.152000   
convnext_tiny.fb_in22k_ft_in1k             CLASSIC_AT      0.693000  0.674000   
resnet50.a1_in1k                           TRADES_v2       0.055000  0.000000   
vit_small_patch16_224.augreg_in21k         TRADES_v2       0.022667  0.005000   
vit_small_patch16_224.augreg_in1k          TRADES_v2       0.049000  0.026000   
coatnet_0_rw_224.sw_in1k                   TRADES_v2       0.000000  0.000000   
resnet50.a1_in1k                           CLASSIC_AT      0.083000  0.000000   
vit_small_patch16_224.augreg_in1k          CLASSIC_AT      0.144000  0.151000   
eva02_tiny_patch14_224.mim_in22k           CLASSIC_AT      0.134000  0.140000   
vit_small_patch16_224.augreg_in21k         CLASSIC_AT      0.173000  0.185333   
eva02_tiny_patch14_224.mim_in22k           TRADES_v2       0.130667  0.135000   
robust_deit_small_patch16_224              TRADES_v2       0.005000  0.120000   
vit_small_patch16_224.augreg_in21k_ft_in1k CLASSIC_AT      0.137000  0.138000   
vit_small_patch16_224.dino                 CLASSIC_AT      0.078667  0.078667   
convnext_tiny.fb_in22k                     TRADES_v2       0.000000  0.000000   
convnext_tiny.fb_in1k                      CLASSIC_AT      0.092000  0.092000   
convnext_tiny.fb_in22k_ft_in1k             TRADES_v2       0.010000  0.010000   
vit_small_patch16_224.augreg_in21k_ft_in1k TRADES_v2            NaN       NaN   
coatnet_0_rw_224.sw_in1k                   CLASSIC_AT      0.134000  0.131000   
convnext_tiny.fb_in22k                     CLASSIC_AT           NaN       NaN   
vit_small_patch16_224.dino                 TRADES_v2       0.028000  0.028000   
swin_tiny_patch4_window7_224.ms_in1k       CLASSIC_AT           NaN       NaN   

dataset                                                                       \
metric                                                    Linf_acc clean_acc   
backbone                                   loss_function                       
robust_convnext_tiny                       CLASSIC_AT     0.703000  0.810000   
robust_resnet50                            CLASSIC_AT     0.646000  0.810000   
robust_convnext_tiny                       TRADES_v2      0.800000  0.948000   
swin_tiny_patch4_window7_224.ms_in1k       TRADES_v2      0.096000  0.902000   
convnext_tiny.fb_in1k                      TRADES_v2      0.000000  0.945000   
deit_small_patch16_224.fb_in1k             TRADES_v2      0.185000  0.724000   
robust_resnet50                            TRADES_v2      0.122000  0.150000   
deit_small_patch16_224.fb_in1k             CLASSIC_AT     0.354667  0.786667   
robust_deit_small_patch16_224              CLASSIC_AT     0.143000  0.167000   
convnext_tiny.fb_in22k_ft_in1k             CLASSIC_AT     0.543000  0.823000   
resnet50.a1_in1k                           TRADES_v2      0.000000  0.803000   
vit_small_patch16_224.augreg_in21k         TRADES_v2      0.000000  0.795000   
vit_small_patch16_224.augreg_in1k          TRADES_v2      0.006000  0.671000   
coatnet_0_rw_224.sw_in1k                   TRA

In [14]:
df_sorted

,,caltech101_L1_acc,caltech101_L2_acc,caltech101_Linf_acc,caltech101_clean_acc,caltech101_common_acc,caltech101_geom,caltech101_model_size,caltech101_sum,fgvc-aircraft-2013b_L1_acc,fgvc-aircraft-2013b_L2_acc,...,uc-merced-land-use-dataset_Linf_acc,uc-merced-land-use-dataset_clean_acc,uc-merced-land-use-dataset_common_acc,uc-merced-land-use-dataset_geom,uc-merced-land-use-dataset_model_size,uc-merced-land-use-dataset_sum,TOTAL_score_sum,TOTAL_score_geom,TOTAL_nan_sum_cnt,TOTAL_nan_geom_cnt
backbone,loss_function,,,,,,,,,,,,,,,,,,,,,
robust_convnext_tiny,CLASSIC_AT,0.502000,0.671000,0.703000,0.810000,0.720000,1.381017e-01,1,3.406000,0.001000,0.013000,...,0.009524,0.100000,0.104762,0.000000e+00,1,0.220635,7.729302,1.430160e-01,0,0
robust_resnet50,CLASSIC_AT,0.140000,0.427000,0.646000,0.810000,0.606000,1.895597e-02,1,2.629000,0.000000,0.000000,...,0.561905,0.838095,0.550000,2.169440e-02,1,2.597619,6.600286,4.065200e-02,0,0
robust_convnext_tiny,TRADES_v2,0.307000,0.698000,0.800000,0.948000,0.861000,1.399250e-01,1,3.614000,0.003000,0.007000,...,0.139683,0.555556,0.361905,4.585195e-05,1,1.161905,6.071905,1.399713e-01,0,0
swin_tiny_patch4_window7_224.ms_in1k,TRADES_v2,0.342000,0.152000,0.096000,0.902000,0.764000,3.439068e-03,1,2.256000,0.010000,0.010000,...,0.000000,0.283333,0.207143,0.000000e+00,1,0.490476,5.682810,3.439069e-03,0,0
convnext_tiny.fb_in1k,TRADES_v2,0.000000,0.000000,0.000000,0.945000,0.838000,0.000000e+00,1,1.783000,0.000000,0.000000,...,0.000000,0.419048,0.307143,0.000000e+00,1,0.726190,5.481524,0.000000e+00,1,1
deit_small_patch16_224.fb_in1k,TRADES_v2,0.421333,0.375000,0.185000,0.724000,0.521333,1.103273e-02,1,2.226667,0.000000,0.000000,...,0.000000,0.630952,0.457143,0.000000e+00,1,1.088095,5.391762,1.103273e-02,0,0
robust_resnet50,TRADES_v2,0.088000,0.114000,0.122000,0.150000,0.137000,2.515123e-05,1,0.611000,0.000000,0.002000,...,0.121429,0.185714,0.142857,9.971540e-06,1,0.576190,4.334524,1.400942e-04,0,0
deit_small_patch16_224.fb_in1k,CLASSIC_AT,0.548000,0.538000,0.354667,0.786667,0.670000,5.511233e-02,1,2.897333,0.012000,0.010000,...,0.009524,0.311905,0.266667,4.728585e-06,1,0.745238,3.909571,5.511707e-02,0,0
robust_deit_small_patch16_224,CLASSIC_AT,0.153000,0.152000,0.143000,0.167000,0.155000,8.608336e-05,1,0.770000,0.009000,0.048000,...,0.002381,0.045238,0.045238,0.000000e+00,1,0.092857,3.802190,5.409106e-03,0,0


In [15]:
# Flatten the columns
df_sorted.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_sorted.columns]


df_reset = df_sorted.reset_index()
sub_df = df_reset[["backbone", "loss_function", "TOTAL_score_sum"]]

# Now access the columns

sub_df

,backbone,loss_function,TOTAL_score_sum
0,robust_convnext_tiny,CLASSIC_AT,7.729302
1,robust_resnet50,CLASSIC_AT,6.600286
2,robust_convnext_tiny,TRADES_v2,6.071905
3,swin_tiny_patch4_window7_224.ms_in1k,TRADES_v2,5.682810
4,convnext_tiny.fb_in1k,TRADES_v2,5.481524
5,deit_small_patch16_224.fb_in1k,TRADES_v2,5.391762
6,robust_resnet50,TRADES_v2,4.334524
7,deit_small_patch16_224.fb_in1k,CLASSIC_AT,3.909571
8,robust_deit_small_patch16_224,CLASSIC_AT,3.802190
9,convnext_tiny.fb_in22k_ft_in1k,CLASSIC_AT,3.655667
